# Workflow

In [ ]:
import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

main_folder = './MNE-eegbci-data/files/eegmmidb/1.0.0'

subdirectories = [f.path for f in os.scandir(main_folder) if f.is_dir()]

opened_files = []
closed_files = []

for subdirectory in subdirectories:
    files = os.listdir(subdirectory)
    
    if len(files) > 0:
        for file in files:
            
            if file[-6:] == '01.edf':
                # This is data for eyes opened
                eyes_opened = os.path.join(subdirectory, file)
                print(eyes_opened)
                opened_files.append(eyes_opened)

            if file[-6:] == '02.edf':
                # This is data for eyes closed
                eyes_closed = os.path.join(subdirectory, file)
                print(eyes_closed)
                closed_files.append(eyes_closed)
    else:
        print(f"No files found in {subdirectory}")

## Filters

In [ ]:
large_open_data = []
# This is a list of all the data for eyes opened
large_closed_data = []
# This is a list of all the data for eyes closed

# for data in opened_files:
#     large_open_data.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))

for data in closed_files:
    large_closed_data.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))

for data in opened_files[:10]:
    data1 = mne.io.read_raw_edf(data, preload=True, verbose=False)
    data1.compute_psd().plot()
    data1.filter(0.1, 40.0, fir_design="firwin", skip_by_annotation="edge", verbose=False)

    # low_cut = 0.1
    # high_cut = 30
    # data_copy = data1.copy().filter(low_cut, high_cut)
    # data_copy.compute_psd().plot()

    # ica = mne.preprocessing.ICA(n_components=45, random_state=97, max_iter=800, verbose=False)
    # ica.fit(data1, verbose=False)
    # ica.exclude = [1, 2]
    # data1 = ica.apply(data1, verbose=False)

    good_channels1 = data1.info['ch_names']

    eeg_data1 = data1.get_data(picks=good_channels1)

    large_open_data.append(eeg_data1)

In [ ]:
plt.plot(np.array(large_open_data[0])[0, :100])
plt.show()

In [ ]:
for data in closed_files[:10]:
    data1 = mne.io.read_raw_edf(data, preload=True, verbose=False)
    data1.compute_psd().plot()
    data1.filter(0.1, 40.0, fir_design="firwin", skip_by_annotation="edge", verbose=False)

    #low_cut = 0.1
    #high_cut = 30
    #data_copy = data1.copy().filter(low_cut, high_cut)
    #data_copy.compute_psd().plot()

    # ica = mne.preprocessing.ICA(n_components=45, random_state=97, max_iter=800, verbose=False)
    # ica.fit(data1, verbose=False)
    # ica.exclude = [1, 2]
    # data1 = ica.apply(data1, verbose=False)

    good_channels1 = data1.info['ch_names']

    eeg_data1 = data1.get_data(picks=good_channels1)

    large_open_data.append(eeg_data1)